## loss (第二种)


$$ loss = \frac{1}{n} \sum{|(kx_i + b_i) - y_i|}  $$

In [3]:
def loss(y, y_hat): # to evaluate the performance 预测值与真实值的差别
    
    return sum(((y_i - y_hat_i)**2)**(1/2) for y_i,y_hat_i in zip(list(y),list(y_hat))) / len(y)

In [4]:
import random

In [5]:
def price(rm, k, b):
    """f(x) = k * x + b"""
    return k * rm + b 

In [6]:
from sklearn.datasets import load_boston

In [8]:
data = load_boston()

In [9]:
X, y = data['data'],data['target']

In [10]:
X_rm = X[:,5]

In [11]:
def partial_k2(x, y, y_hat):
    n = len(y) 
    out = 0
    for x_i,y_i,y_hat_i in zip(list(X_rm),list(y),list(y_hat)):
        if y_i - y_hat_i >= 0:
            out += 1/n * (-x_i)
        else:
            out += 1/n * x_i
    return out

In [12]:
def partial_b2(x, y, y_hat):
    n = len(y)
    out = 0
    for x_i,y_i,y_hat_i in zip(list(X_rm),list(y),list(y_hat)):
        if y_i - y_hat_i >= 0:
            out += -1/n 
        else:
            out += 1/n
    return out

In [14]:
trying_times = 2000

min_loss = float('inf')
best_k,best_b = None,None

current_k = random.random() * 200 - 100
current_b = random.random() * 200 - 100

learning_rate = 1e-03
for i in range(trying_times):
        
    price_by_k_and_b = [price(r, current_k, current_b) for r in X_rm]
    
    current_loss = loss(y,price_by_k_and_b)
    if current_loss < min_loss: # performance became better
        min_loss = current_loss
        if i % 50 == 0:
            print('When time is: {}, get best_k: {} best_b: {}, and the loss is {}'.format(i,best_k,best_b,min_loss))
    
    k_gradient = partial_k2(X_rm, y, price_by_k_and_b)
    b_gradient = partial_b2(X_rm, y, price_by_k_and_b)
    
    current_k -= k_gradient*learning_rate
    current_b -= b_gradient*learning_rate

When time is: 0, get best_k: None best_b: None, and the loss is 573.8546166120326
When time is: 50, get best_k: None best_b: None, and the loss is 571.8297851428968
When time is: 100, get best_k: None best_b: None, and the loss is 569.8049536737616
When time is: 150, get best_k: None best_b: None, and the loss is 567.780122204626
When time is: 200, get best_k: None best_b: None, and the loss is 565.75529073549
When time is: 250, get best_k: None best_b: None, and the loss is 563.730459266354
When time is: 300, get best_k: None best_b: None, and the loss is 561.7056277972194
When time is: 350, get best_k: None best_b: None, and the loss is 559.6807963280837
When time is: 400, get best_k: None best_b: None, and the loss is 557.655964858948
When time is: 450, get best_k: None best_b: None, and the loss is 555.6311333898125
When time is: 500, get best_k: None best_b: None, and the loss is 553.606301920677
When time is: 550, get best_k: None best_b: None, and the loss is 551.5814704515413
W

## Dynamic Programming 动态规划 (不断查表)

可能遇到和复杂的情况，搜索的点很多，就会导致生成的树很大。
会出现重复搜索的情况。


In [135]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

In [136]:
from collections import defaultdict

In [137]:
price = defaultdict(int)  # price字典对于未定义的key对应的value是0

In [138]:
for i,p in enumerate(original_price):
    price[i+1] = p 

In [69]:
some_default_dict = defaultdict(lambda : 'None')  # ()里表示的是未设置过的key的value默认值

## Get the max splitting by enumerate

假如想要知道函数运行的时间又多长，或者看这个函数被调用多少次
就要用 装饰器

In [139]:
def r(n):  # n米长的钢管，最优的价格是多少
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)] #这个+号，是对列表进行操作，相当于append
    )

In [140]:
r(10) # 当n变大时，运行时间会大大加长

30

In [12]:
called_time = defaultdict(int)

def get_call_times(f):
    result = f() 
    print('function: {} called once!'.format(f.__name__))
    called_time[f.__name__] += 1
    return result

In [13]:
def some_function_1(): print('I am function 1')

面向时什么意思？
面向x 
那么x时可以作为一个变量的，可以作为函数输入，输出，运算单元

In [14]:
get_call_times(some_function_1)

I am function 1
function: some_function_1 called once!


In [15]:
called_time

defaultdict(int, {'some_function_1': 1})

In [19]:
from collections import Counter

In [16]:
call_time_with_arg = defaultdict(int)

In [17]:
def r(n):  # n米长的钢管，最优的价格是多少
    fname = r.__name__
    call_time_with_arg[(fname, n)] += 1
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)] #这个+号，是对列表进行操作，相当于append
    )

In [141]:
r(15)

45

In [142]:
Counter(call_time_with_arg).most_common()  # 看运行上面的r(15)，统计各个r(n)都运行了几次
# 可以看到很多r(n)都被重复计算了很多次，这就浪费了很多时间

[(('r', 1), 3188646),
 (('r', 2), 1062882),
 (('r', 3), 354294),
 (('r', 4), 118098),
 (('r', 5), 39366),
 (('r', 6), 13122),
 (('r', 7), 4374),
 (('r', 8), 1458),
 (('r', 9), 486),
 (('r', 10), 162),
 (('r', 11), 54),
 (('r', 12), 18),
 (('r', 13), 6),
 (('r', 14), 2),
 (('r', 15), 1)]

## 装饰器

In [57]:
def r(n):  # n米长的钢管，最优的价格是多少
    
    # 利用装饰器 可以省略下面这两句话
    # 这两句话只是观察的时候，想要运行的，并不是真正想要运行的
    # fname = r.__name__
    # call_time_with_arg[(fname, n)] += 1
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)] #这个+号，是对列表进行操作，相当于append
    )

In [143]:
called_time_with_arg = defaultdict(int)

def get_call_time(f):
    def wrap(n):
#         print('I can count')
        result = f(n)
        called_time_with_arg[(f.__name__, n)] += 1
        return result
    return wrap

In [144]:
def add_ten(n): return n+10

In [145]:
add_ten(10)

20

In [146]:
add_ten_with_call_time = get_call_time(add_ten)

In [147]:
add_ten_with_call_time   # 这是一个函数

<function __main__.get_call_time.<locals>.wrap>

In [148]:
add_ten_with_call_time(9)   # 将10作为n代入了wrap。即就相当于out[31]加上(10)

19

In [149]:
called_time_with_arg

defaultdict(int, {('add_ten', 9): 1})

In [150]:
add_ten = get_call_time(add_ten)

In [151]:
add_ten(8)

18

上面这种写法可以换成用装饰器的方法

In [152]:
@get_call_time  # 这句话就表示 add_ten = get_call_time(add_ten)
def add_twenty(n):
    return n + 20
# 加这个 @get_call_time 就相当于上面那种调用的写法
# 相当于 get_call_time(add_twenty)

In [153]:
add_twenty(10)

30

所以回到计算最佳价格的r(n)函数，添加装饰器

In [76]:
called_time_with_arg = defaultdict(int)

In [154]:
@get_call_time
def r(n):  # n米长的钢管，最优的价格是多少
    """
    Args: n is the iron length
    Return: the max revenue
    """
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)] #这个+号，是对列表进行操作，相当于append
    )

In [157]:
called_time_with_arg = defaultdict(int)
r(10)

30

In [158]:
called_time_with_arg

defaultdict(int,
            {('r', 1): 13122,
             ('r', 2): 4374,
             ('r', 3): 1458,
             ('r', 4): 486,
             ('r', 5): 162,
             ('r', 6): 54,
             ('r', 7): 18,
             ('r', 8): 6,
             ('r', 9): 2,
             ('r', 10): 1})

In [159]:
from functools import wraps  # 解决失去函数命名的问题

In [160]:
called_time_with_arg = defaultdict(int)

def get_call_time(f):
    @wraps(f)
    def wrap(n):
        result = f(n)
        called_time_with_arg[(f.__name__, n)] += 1
        return result
    return wrap

In [161]:
r = get_call_time(r)
help(r)

Help on function wrap in module __main__:

wrap(n)



既然重复计算了很多个值，就希望能将这些值保存起来，下次再遇到的时候，直接读取

In [162]:
cache = {}  # 缓存
def memo(f): # 不用重复计算，直接调用之前计算过的结果
    
    @wraps(f)
    def _wrap(arg):
        result = None
        if arg in cache: 
            result = cache[arg]
        else:    
            result = f(arg)
            cache[arg] = result
        return result
    return _wrap

In [163]:
@get_call_time
@memo
def r(n):  # n米长的钢管，最优的价格是多少
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)] #这个+号，是对列表进行操作，相当于append
    )

In [164]:
called_time_with_arg = defaultdict(int)
r(19)

57

In [165]:
called_time_with_arg

defaultdict(int,
            {('r', 1): 36,
             ('r', 2): 34,
             ('r', 3): 32,
             ('r', 4): 30,
             ('r', 5): 28,
             ('r', 6): 26,
             ('r', 7): 24,
             ('r', 8): 22,
             ('r', 9): 20,
             ('r', 10): 18,
             ('r', 11): 16,
             ('r', 12): 14,
             ('r', 13): 12,
             ('r', 14): 10,
             ('r', 15): 8,
             ('r', 16): 6,
             ('r', 17): 4,
             ('r', 18): 2,
             ('r', 19): 1})

In [166]:
cache

{1: 1,
 2: 5,
 3: 8,
 4: 10,
 5: 13,
 6: 17,
 7: 18,
 8: 22,
 9: 25,
 10: 30,
 11: 35,
 12: 36,
 13: 40,
 14: 43,
 15: 45,
 16: 48,
 17: 52,
 18: 53,
 19: 57}

现在我只知道可以得到的最优的价格，并不知道如何切分这跟钢管。

In [167]:
solution = {}

In [168]:
@memo
def r(n):  # n米长的钢管，最优的价格是多少
    max_price, max_split = max(
        [(price[n],0)] + [(r(i) + r(n-i),i) for i in range(1,n)], key = lambda x:x[0]
    )
    
    solution[n] = (n - max_split, max_split)
    
    return max_price

In [169]:
cache = {}
r(38)

118

In [170]:
 solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3),
 21: (11, 10),
 22: (11, 11),
 23: (22, 1),
 24: (22, 2),
 25: (22, 3),
 26: (24, 2),
 27: (25, 2),
 28: (22, 6),
 29: (28, 1),
 30: (28, 2),
 31: (28, 3),
 32: (22, 10),
 33: (22, 11),
 34: (33, 1),
 35: (33, 2),
 36: (33, 3),
 37: (35, 2),
 38: (36, 2)}

In [171]:
def parse_solution(n):
    left_split, right_split = solution[n]
    if right_split == 0: return [left_split]
    return parse_solution(left_split)+parse_solution(right_split)

In [172]:
parse_solution(32)

[11, 11, 10]

## Problem 2 : Eidt Distance
计算两个单词之间的相似度
+ insertion
+ delection
+ substitution

应用范围：
+ 拼写错误识别


In [209]:
solution = defaultdict(None)

In [210]:
from functools import lru_cache

In [211]:
@lru_cache(maxsize = 2**10)
def edit_distance(string1, string2):
    # 如果其中一个字符串长度为0，则返回距离：为另一个字符串的长度
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    # 取最后一个字符
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        #string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))
        
    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key = lambda x:x[0])
    
    solution[(string1, string2)] = operation
    
    return min_distance
    

In [212]:
edit_distance('ABCDE', 'ABCCEF')

2

In [214]:
solution

defaultdict(None,
            {('A', 'A'): '',
             ('A', 'AB'): 'ADD B',
             ('A', 'ABC'): 'ADD C',
             ('A', 'ABCC'): 'ADD C',
             ('A', 'ABCCE'): 'ADD E',
             ('A', 'ABCCEF'): 'ADD F',
             ('AB', 'A'): 'DEL B',
             ('AB', 'AB'): '',
             ('AB', 'ABC'): 'ADD C',
             ('AB', 'ABCC'): 'ADD C',
             ('AB', 'ABCCE'): 'ADD E',
             ('AB', 'ABCCEF'): 'ADD F',
             ('ABC', 'A'): 'DEL C',
             ('ABC', 'AB'): 'DEL C',
             ('ABC', 'ABC'): '',
             ('ABC', 'ABCC'): 'ADD C',
             ('ABC', 'ABCCE'): 'ADD E',
             ('ABC', 'ABCCEF'): 'ADD F',
             ('ABCD', 'A'): 'DEL D',
             ('ABCD', 'AB'): 'DEL D',
             ('ABCD', 'ABC'): 'DEL D',
             ('ABCD', 'ABCC'): 'SUB D => C',
             ('ABCD', 'ABCCE'): 'ADD E',
             ('ABCD', 'ABCCEF'): 'ADD F',
             ('ABCDE', 'A'): 'DEL E',
             ('ABCDE', 'AB'): 'DEL E',
   

In [224]:
def parse_solution(s1, s2):
    method = []
    temp = solution[(s1, s2)]
    while s1 != s2:
        temp = solution[(s1, s2)]
        if temp != '':
            method.append(temp)
        if temp[:3] == 'ADD':
            s2 = s2[:-1]
        elif temp[:3] == 'DEL':
            s1 = s2[:-1]
        elif temp[:3] == 'SUB':
            s1 = s1[:-1]
            s1 += temp[-1]
        else:
            s1 = s1[:-1]
            s2 = s2[:-1]
    method = method[::-1]
    return method
        

In [225]:
print(parse_solution('ABC', 'ABCCEF'))

['ADD C', 'ADD E', 'ADD F']


## Part 5-1

1. Why do we use Derivative / Gradient to fit s target function?
求梯度可以找到使目标函数下降最快的方向

2. In the words' Gradient Descent', what's the Gradient and what's the Descent?
gradient指的是梯度，就是目标函数分别对每个模型的参数求偏导得到的导数。
descent指的是沿着梯度的反方向，目标函数的值会变小，逐渐收敛。

3. What's the advantages of the 3rd gradient descent method compared to the previous methods?
下降的方向更准确，下降的速度更快。

4. Using the simple words to describe: What's the machine leanring.
建造一个模型，经过不断的调参过程，使得模型输出的结果越来越准确。

## Part 5

1. Why do we need dynamic programming? What's the difference of dynamic programming and previous talked search problem?
需要动态规划的原因：减少重复的计算，通过缓存、查表的方式，获得已经计算过的值。

2. Why do we still need dynamic programming? Why not we train a machine learning to fit a function which could get the right answer based on inputs?
